In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import datetime

Proposed Datasets
Show Info:
Season
Episode
Air date
Contestant 1
Contestant 2
Contestant 3
Returning Contestant

Show Details:
Episode
Round
Category
Clue
Value
Clue Order
Correct Response
Correct Responder
Incorrect Response 1
Incorrect Responder 1
Incorrect Response 2
Incorrect Responder 2
Incorrect Response 3
Incorrect Responder 3
Daily Double
Wager

Show Statistics:
J Round Score C1
J Round Score C2
J Round Score C3
DJ Round Score C1
DJ Round Score C2
DJ Round Score C3
FJ Wager C1
FJ Wager C2
FJ Wager C3
FJ Round Score C1
FJ Round Score C2
FJ Round Score C3


All Jeopardy! seasons:
http://www.j-archive.com/listseasons.php

## Season List

In [2]:
response = requests.get('http://www.j-archive.com/listseasons.php')

In [3]:
soup = BeautifulSoup(response.text)
#soup

In [4]:
season_urls = np.array(['http://www.j-archive.com/' + a['href'] for a in soup.findAll('a') 
                    if 'showseason' in a['href'] and 'http' not in a['href']])
season_urls

array(['http://www.j-archive.com/showseason.php?season=37',
       'http://www.j-archive.com/showseason.php?season=goattournament',
       'http://www.j-archive.com/showseason.php?season=36',
       'http://www.j-archive.com/showseason.php?season=35',
       'http://www.j-archive.com/showseason.php?season=34',
       'http://www.j-archive.com/showseason.php?season=33',
       'http://www.j-archive.com/showseason.php?season=32',
       'http://www.j-archive.com/showseason.php?season=31',
       'http://www.j-archive.com/showseason.php?season=30',
       'http://www.j-archive.com/showseason.php?season=29',
       'http://www.j-archive.com/showseason.php?season=28',
       'http://www.j-archive.com/showseason.php?season=27',
       'http://www.j-archive.com/showseason.php?season=26',
       'http://www.j-archive.com/showseason.php?season=25',
       'http://www.j-archive.com/showseason.php?season=24',
       'http://www.j-archive.com/showseason.php?season=23',
       'http://www.j-archive

In [5]:
seasons = pd.DataFrame(season_urls)
seasons.columns = ['season_url']
#seasons
seasons.to_csv('data/j_seasons.csv', index=False)

## Episode List

In [6]:
response = requests.get('http://www.j-archive.com/showseason.php?season=37')

In [7]:
soup = BeautifulSoup(response.text)
#soup

In [8]:
episodes = np.array([a['href'] for a in soup.findAll('a') if 'game_id' in a['href']])
episodes = pd.DataFrame(episodes)
episodes.columns = ['episode_url']

In [9]:
episodes.to_csv('data/j_episodes.csv', index=False)

## Episode Details

In [63]:
#response = requests.get('http://www.j-archive.com/showgame.php?game_id=6895')
response = requests.get('http://www.j-archive.com/showgame.php?game_id=6893')


In [64]:
soup = BeautifulSoup(response.text)
#soup

In [65]:
header = soup.find('h1')
header = header.text
show_num = header.split('-')[0].strip().split('#')[1]
air_date = datetime.strftime(datetime.strptime(header.split('-')[1].strip().split(',',1)[1].strip(),'%B %d, %Y'), '%Y-%m-%d')
#print(show_num, ', ', air_date)

In [66]:
season = soup.find('a', text='[current season]')
print(season)
season = season['href'].split('=')[1]

<a href="http://www.j-archive.com/showseason.php?season=37">[current season]</a>


In [67]:
scores = soup.findAll(['h3','table'])
score_texts = np.array([['Scores at the end of the Jeopardy! Round:', 'J'],
                        ['Scores at the end of the Double Jeopardy! Round:', 'DJ'],
                        ['Final scores:', 'FJ']])
j_round = []
j_name = []
j_score = []
for x in score_texts:
    for element in scores:
        if element.text == x[0]:
            while element.name != 'table':
                element = element.next_sibling
                #print('name: ',element.name)
            score_table = element.findAll('tr')
            [j_round.append(x[1]) for td in score_table[0].findAll('td')]
            [j_name.append(td.text) for td in score_table[0].findAll('td')]
            [j_score.append(td.text) for td in score_table[1].findAll('td')]

j_scores = pd.DataFrame({'show_num': show_num, 'round_code': j_round, 'contestant': j_name, 'score': j_score})
j_scores

,show_num,round_code,contestant,score
0,8303,J,Brayden,"$7,600"
1,8303,J,Rhonda,"$1,400"
2,8303,J,Spencer,"$5,000"
3,8303,DJ,Brayden,"$31,800"
4,8303,DJ,Rhonda,"$2,200"
5,8303,DJ,Spencer,"$7,400"
6,8303,FJ,Brayden,"$35,000"
7,8303,FJ,Rhonda,"$4,399"
8,8303,FJ,Spencer,"$4,401"


In [68]:
remarks_text = 'Coryat scores:'
j_right = []
j_ddright = []
j_wrong = []
j_ddwrong = []
for element in scores:
    if element.text == remarks_text:
        while element.name != 'table':
            element = element.next_sibling
        remarks_table = element.findAll('td', class_ = 'score_remarks')
        [j_right.append((re.findall('(\d{1,2}) R', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_ddright.append((re.findall('(\d{1}) DD', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_wrong.append((re.findall('(\d{1,2}) W', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]
        [j_ddwrong.append((re.findall('(\d{1}) DD', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]

j_remarks = pd.DataFrame({'contestant': j_name[:3], 'num_right': j_right, 'num_ddright': j_ddright, 'num_wrong': j_wrong,
                         'num_ddwrong': j_ddwrong})
j_remarks

,contestant,num_right,num_ddright,num_wrong,num_ddwrong
0,Brayden,28,3,1,0
1,Rhonda,7,0,2,0
2,Spencer,15,0,2,0


In [69]:
j_scores = j_scores.merge(j_remarks, how = 'left', on='contestant')
j_scores

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong
0,8303,J,Brayden,"$7,600",28,3,1,0
1,8303,J,Rhonda,"$1,400",7,0,2,0
2,8303,J,Spencer,"$5,000",15,0,2,0
3,8303,DJ,Brayden,"$31,800",28,3,1,0
4,8303,DJ,Rhonda,"$2,200",7,0,2,0
5,8303,DJ,Spencer,"$7,400",15,0,2,0
6,8303,FJ,Brayden,"$35,000",28,3,1,0
7,8303,FJ,Rhonda,"$4,399",7,0,2,0
8,8303,FJ,Spencer,"$4,401",15,0,2,0


In [70]:
finish_order = []
winner = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[0]
second = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[1]
third = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[2]
finish_order.append([winner, 'Winner'])
finish_order.append([second, 'Second'])
finish_order.append([third, 'Third'])
finish_order = pd.DataFrame(finish_order)
finish_order.columns=(['contestant', 'finish'])
j_scores = j_scores.merge(finish_order, how = 'left', on = 'contestant')
j_scores

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong,finish
0,8303,J,Brayden,"$7,600",28,3,1,0,Third
1,8303,J,Rhonda,"$1,400",7,0,2,0,Second
2,8303,J,Spencer,"$5,000",15,0,2,0,Winner
3,8303,DJ,Brayden,"$31,800",28,3,1,0,Third
4,8303,DJ,Rhonda,"$2,200",7,0,2,0,Second
5,8303,DJ,Spencer,"$7,400",15,0,2,0,Winner
6,8303,FJ,Brayden,"$35,000",28,3,1,0,Third
7,8303,FJ,Rhonda,"$4,399",7,0,2,0,Second
8,8303,FJ,Spencer,"$4,401",15,0,2,0,Winner


In [71]:
categories = soup.findAll('td', attrs = {'class': 'category_name'})

In [72]:
categories = np.array([x.text for x in categories])
categories

array(['DOUBLE "O" 7-LETTER WORDS', 'TV COMEDIES', 'THE COUNTRY SHE LED',
       'U.S. FACT SHEET', 'THIS & THAT', 'GINGERBREAD', 'CAVE CAVEATS',
       'MINCED OATHS', 'BOATS & SHIPS', 'MEDICAL TERMS', 'ENDING',
       'ON A HIGH NOTE', 'ROYALTY'], dtype='<U25')

In [73]:
rounds = []
[[rounds.append(item) for x in range(6)] for item in ['J', 'DJ']]
rounds.append('FJ')
rounds = np.array(rounds)
rounds

array(['J', 'J', 'J', 'J', 'J', 'J', 'DJ', 'DJ', 'DJ', 'DJ', 'DJ', 'DJ',
       'FJ'], dtype='<U2')

In [74]:
catnums = np.array(['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6', '1'])
catnums

array(['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6', '1'],
      dtype='<U1')

In [75]:
categories = pd.DataFrame({'category': categories, 'round_code': rounds, 'cat_num': catnums})

In [76]:
categories['season'] = season
categories['show_num'] = show_num
categories['air_date'] = air_date
categories

,category,round_code,cat_num,season,show_num,air_date
0,"DOUBLE ""O"" 7-LETTER WORDS",J,1,37,8303,2020-12-16
1,TV COMEDIES,J,2,37,8303,2020-12-16
2,THE COUNTRY SHE LED,J,3,37,8303,2020-12-16
3,U.S. FACT SHEET,J,4,37,8303,2020-12-16
4,THIS & THAT,J,5,37,8303,2020-12-16
5,GINGERBREAD,J,6,37,8303,2020-12-16
6,CAVE CAVEATS,DJ,1,37,8303,2020-12-16
7,MINCED OATHS,DJ,2,37,8303,2020-12-16
8,BOATS & SHIPS,DJ,3,37,8303,2020-12-16
9,MEDICAL TERMS,DJ,4,37,8303,2020-12-16


In [77]:
#split the results by the "clue" class
clues = soup.find_all('td', attrs = {'class': 'clue'})
print(type(clues))
print(type(soup))

<class 'bs4.element.ResultSet'>
<class 'bs4.BeautifulSoup'>


In [78]:
all_rows = []
for clue in clues:
    #print(type(clue))
    new_row = []
    #Get our clue number, which tells us the round, category, and board position
    #!!!!!!!!!!!!!!!!!!!!!!Check to be sure the clue has an id!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    clue_id = clue.find('td', class_='clue_text')['id']
    #What round is this?
    round_code = clue_id.split('_')[1]
    cat_num = clue_id.split('_')[2] if not round_code == 'FJ' else 'checkme'
    clue_num = clue_id.split('_')[3] if not round_code == 'FJ' else 'checkme'
    if not round_code == 'FJ':
        #Get our clue value
        #DD does not have a clue_value, it has a clue_value_daily_double
        #For DD split out the dollar value and set the flag
        clue_value = clue.find('td', class_='clue_value_daily_double').text.split(' ')[1] if not clue.find('td', {'class':'clue_value'}) else clue.find('td', {'class':'clue_value'}).text
        daily_double = False if not clue.find('td', class_='clue_value_daily_double') else True
        clue_order = clue.find('td', class_='clue_order_number').text
        clue_text = clue.find('td', class_='clue_text').text
        corr_resp = BeautifulSoup(clue.find('div', onmouseover=True).get('onmouseover'))
        #print(corr_resp)
        #Check for clues with no correct responder, and set the flag
        corr_respr1 = '' if not corr_resp.find('td', class_='right') else corr_resp.find('td', class_='right').text
        corr_FJ_wager1 = ''
        corr_respr2 = ''
        corr_FJ_wager2 = ''
        corr_respr3 = ''
        corr_FJ_wager3 = ''
        triple_stumper = True if not corr_resp.find('td', class_='right') else False
        incorr_resprs = corr_resp.findAll('td', class_='wrong')
        #Okay, THIS is ugly
        incorr_respr1 = ''
        incorr_FJ_wager1 = ''
        incorr_respr2 = ''
        incorr_FJ_wager2 = ''
        incorr_respr3 = ''
        incorr_FJ_wager3 = ''
        if len(incorr_resprs) >= 1:
            incorr_respr1 = '' if incorr_resprs[0].text == 'Triple Stumper' else incorr_resprs[0].text
        if len(incorr_resprs) >= 2:
            incorr_respr2 = '' if incorr_resprs[1].text == 'Triple Stumper' else incorr_resprs[1].text
        if len(incorr_resprs) == 3:
            incorr_respr3 = '' if incorr_resprs[2].text == 'Triple Stumper' else incorr_resprs[2].text
        corr_resp = corr_resp.find('em', class_='correct_response').text
        new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                   corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                   triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3 ]
        all_rows.append(new_row)


TypeError: 'NoneType' object is not subscriptable

In [79]:
all_rows

[['J',
  '1',
  '1',
  'clue_J_1_1',
  '$200',
  False,
  '6',
  'On commercial airlines, exit rows & bulkhead seats typically offer extra this',
  'legroom',
  'Spencer',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '2',
  '1',
  'clue_J_2_1',
  '$200',
  False,
  '8',
  'In 2001 this "Friends" character married Chandler Bing',
  'Monica',
  'Spencer',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '3',
  '1',
  'clue_J_3_1',
  '$200',
  False,
  '25',
  'Isabel Peron',
  'Argentina',
  'Brayden',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '4',
  '1',
  'clue_J_4_1',
  '$200',
  False,
  '24',
  "At 77 million cubic feet, this Virginia office building is the country's largest",
  'the Pentagon',
  'Rhonda',
  '',
  '',
  '',
  '',
  '',
  False,
  '',
  '',
  '',
  '',
  '',
  ''],
 ['J',
  '5',
  '1',
  'clue_J_5_1',
  '$200',
  False,
  '9',
  'A sidewinder is this

In [55]:
round_code = 'FJ'
cat_num = '1'
clue_num = '1'
clue_id = 'clue_FJ'
clue_value = ''
daily_double = False
clue_order = ''
clue_text = ''
corr_resp = ''
triple_stumper = True

final_round = soup.find('table', class_ = 'final_round').find_all('td')
#type(final_round)
#final_round - this will absolutely barf and so will a print; it gives a type error

for item in final_round:
    #this only works because it's the first item! aaargh
    stats = BeautifulSoup(item.find('div', onmouseover=True).get('onmouseover'))
    if stats:
        break
corr_resp = stats.find('em')
corr_resp = corr_resp.get_text()

#make a dictionary of responders
resprs = {}
wagers = stats.findAll('td')

right = 0
wrong = 0
for td in wagers:
    td_text = td.get_text()
    if td.has_attr('class'):
        td_class = td.get('class')[0]
        if td_class == 'right':
            triple_stumper = False
            right +=1
            key_name = 'corr_respr' + str(right)
            resprs[key_name] = td_text
            wager_key = 'corr_FJ_wager' + str(right)
        else:
            wrong +=1
            key_name = 'incorr_respr' + str(wrong)
            resprs[key_name] = td_text
            wager_key = 'incorr_FJ_wager' + str(wrong)
    if td.has_attr('rowspan'):
        continue
    if not td.has_attr('class'):
        resprs[wager_key] = td_text
        
corr_respr1 = resprs.get('corr_respr1', '')
corr_FJ_wager1 = resprs.get('corr_FJ_wager1', '')
corr_respr2 = resprs.get('corr_respr2', '')
corr_FJ_wager2 = resprs.get('corr_FJ_wager2', '')
corr_respr3 = resprs.get('corr_respr3', '')
corr_FJ_wager3 = resprs.get('corr_FJ_wager3', '')
incorr_respr1 = resprs.get('incorr_respr1','')
incorr_FJ_wager1 = resprs.get('incorr_FJ_wager1', '')
incorr_respr2 = resprs.get('incorr_respr2','')
incorr_FJ_wager2 = resprs.get('incorr_FJ_wager2', '')
incorr_respr3 = resprs.get('incorr_respr3','')
incorr_FJ_wager3 = resprs.get('incorr_FJ_wager3', '')

clue_text = soup.find('td', attrs = {'class': 'clue_text', 'id': 'clue_FJ'})
clue_text = clue_text.get_text()
#print(clue_text)

new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                   corr_respr1, corr_FJ_wager1, corr_respr2, corr_FJ_wager2, corr_respr3, corr_FJ_wager3, 
                   triple_stumper, incorr_respr1, incorr_FJ_wager1, incorr_respr2, incorr_FJ_wager2, incorr_respr3, incorr_FJ_wager3]
all_rows.append(new_row)

In [58]:
this_show = pd.DataFrame(all_rows)
this_show.columns = ['round_code', 'cat_num', 'clue_num', 'clue_id', 'clue_value', 'daily_double', 'clue_order', 'clue_text', 'corr_resp', 
                   'corr_respr1', 'corr_FJ_wager1', 'corr_respr2', 'corr_FJ_wager2', 'corr_respr3', 'corr_FJ_wager3', 
                   'triple_stumper', 'incorr_respr1', 'incorr_FJ_wager1', 'incorr_respr2', 'incorr_FJ_wager2', 'incorr_respr3', 'incorr_FJ_wager3']
this_show = this_show.merge(categories, how='left', left_on=['round_code', 'cat_num'], right_on=['round_code', 'cat_num'])
this_show

,round_code,cat_num,clue_num,clue_id,clue_value,daily_double,clue_order,clue_text,corr_resp,corr_respr1,...,incorr_respr1,incorr_FJ_wager1,incorr_respr2,incorr_FJ_wager2,incorr_respr3,incorr_FJ_wager3,category,season,show_num,air_date
0,J,1,1,clue_J_1_1,$200,False,10,"Kern, Imperial, Lassen",California,Devon,...,Brayden,,,,,,STATES BY COUNTY,37,8305,2020-12-18
1,J,2,1,clue_J_2_1,$200,False,23,"Always in style, Justice Ginsburg was famous f...",dissent collar,Amanda,...,,,,,,,RUTH BADER GINSBURG,37,8305,2020-12-18
2,J,3,1,clue_J_3_1,$200,False,30,After winning a 2020 tennis tourney in Aucklan...,the Australian fires,Brayden,...,,,,,,,DONATING THEIR WINNINGS,37,8305,2020-12-18
3,J,4,1,clue_J_4_1,$200,False,28,$2.00 a day was the price when this trailer re...,U-Haul,Brayden,...,,,,,,,MOVING,37,8305,2020-12-18
4,J,5,1,clue_J_5_1,$200,False,4,"Mike Campbell wrote the music for ""Boys Of Sum...",Don Henley,Devon,...,,,,,,,YACHT ROCK,37,8305,2020-12-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,DJ,3,5,clue_DJ_3_5,$2000,False,7,"This playwright's ""The Children's Hour"" was in...",(Lillian) Hellman,Brayden,...,,,,,,,LITERATURE,37,8305,2020-12-18
57,DJ,4,5,clue_DJ_4_5,$2000,False,12,"During this 13-letter process, plants lose wat...",transpiration,Brayden,...,,,,,,,BIOLOGY,37,8305,2020-12-18
58,DJ,5,5,clue_DJ_5_5,$2000,False,9,The only remaining currency in Europe still na...,Liechtenstein,Amanda,...,Brayden,,Devon,,,,MONEY MAKES THE WORLD GO 'ROUND,37,8305,2020-12-18
59,DJ,6,5,clue_DJ_6_5,$2000,False,19,Term for a very old manuscript volume of the s...,a codex,Brayden,...,,,,,,,IN EXCELSIS DEO,37,8305,2020-12-18


In [61]:
this_show.to_csv('data/j_clues.csv', index=False)

In [62]:
#this_show.to_csv('data/j_clues.csv', mode='a', index=False, header=False)

https://github.com/whymarrh/jeopardy-parser/blob/master/parser.py